In [1]:
import sys
sys.path.append("../src")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools
import seaborn as sns
#sns.color_palette('husl', n_colors=20)
from utils.visual_functions import *
import torch
%matplotlib inline
plt.style.use(['science','ieee', 'muted'])
fig_path="../figure/paper/"
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, FuncFormatter)

## CNN Results

In [2]:
gt_truth = {}
predictions = {}
for dataset in ["plaid", "lilac"]:
    pred, gt=get_model_results(dataset=dataset, fig_path=fig_path)
    predictions[dataset]=pred
    gt_truth[dataset]=gt

results for vi image type with plaid dataset
mcc:0.91:0.03
f1:91.67:2.32
zl:8.32:2.35

results for adaptive image type with plaid dataset
mcc:0.98:0.01
f1:97.91:0.97
zl:2.1:0.98

results for plaid
       F1   MCC    ZL
vi  91.82  0.91  8.32

results for vi image type with lilac dataset
mcc:0.85:0.02
f1:85.36:2.04
zl:13.82:1.99

results for adaptive image type with lilac dataset
mcc:0.98:0.01
f1:98.33:0.74
zl:1.66:0.74

results for lilac
       F1   MCC     ZL
vi  85.32  0.85  13.82



## Error analysis

### PLAID dataset

In [3]:
true = gt_truth["plaid"]["adaptive"]
pred = predictions["plaid"]["adaptive"]
image = np.load("../results/plaid_adaptive_50_CNN_1_exp_two_images.npy", allow_pickle=True)
image=np.vstack(image).squeeze()

In [4]:
def idx_to_label(x, labels):
    x_label = []
    for idx in x.tolist():
        x_label.append(labels[idx[0]])
    return x_label

def get_incorrect_idx(y_t, y_p):
    idxs = []
    for idx, l in enumerate(y_p):
        if l!=y_t[idx]:
            idxs.append(idx)
    return idxs
        

In [5]:
pred_label = idx_to_label(pred, list(plaid_labels.values()))
gt_label = idx_to_label(true, list(plaid_labels.values()))
inc_ids = get_incorrect_idx(gt_label, pred_label)

In [ ]:
np.unique(np.array(pred_label))

In [ ]:
for app in np.unique(np.array(pred_label)):
    appidx=np.where(np.array(pred_label)[inc_ids]==app)[0]
    print(f"{app}: {len(appidx)}")


In [ ]:
from IPython import display
i = 1
for idx in np.where(np.array(pred_label)[inc_ids]=="CFL")[0]:
    plt.figure(figsize=set_figure_size(fig_width=2.2))
    plt.matshow(image[inc_ids][idx],cmap='YlGnBu', origin='lower')
    plt.title(f"true:{np.array(gt_label)[inc_ids][idx]}, pred:{np.array(pred_label)[inc_ids][idx]}")
    plt.xlabel("$w$")
    plt.ylabel("$w$")
    plt.xticks([], [])
    ax = plt.gca()
    ax.tick_params(axis="both", which="both", bottom=False, 
               top=False, labelbottom=False, left=False, right=False, labelleft=False)

    savefig(fig_path+f"error_{str(i)}_{np.array(gt_label)[inc_ids][idx]}_{np.array(pred_label)[inc_ids][idx]}_plaid", format=".pdf")
    i+=1
    #plt.show()
    #input("enter")
    #display.clear_output()

In [ ]:
np.array(gt_label)[inc_ids][idx]

In [ ]:
cfl=np.where(np.array(pred_label)[inc_ids]=='Laptop')[0]
fig, axs = plt.subplots(len(cfl)//2,2, sharex=True, figsize=set_figure_size(columns=2))
fig.subplots_adjust(hspace = 0.25)
axs = axs.ravel()
for i, idap in enumerate(cfl):
    axs[i].plot(current[idap], "k--", label="$i(t)$", alpha=0.8)
    #axs[idap].set_title(app)
    axs[i].set_ylabel("Current $(A)$") 
    #axs[idap].set_xlim(0, 501) 
    axs[i].autoscale(tight=True)
    

In [ ]:
current, voltage, labels = load_data("plaid", submetered=False)

## Parameters experiments

### Initilization parameters

In [ ]:
import seaborn as sns
sns.color_palette('husl', n_colors=20)
def get_param_results(dataset="lilac"):
    model_name='CNN'
    width=50
    run_id=1
    cv=4
    multi_dimension=True
    isc=False
    names=lilac_names   if dataset=="lilac"   else plaid_names  
    dataset = dataset+"_isc" if isc else dataset
    image_type= "adaptive"
    baseline = False
    results_cm = {}
    results_f1={}
    results_mcc={}
    results_zl={}
    preds = {}
    trues = {}
    currents = {}
    for delta in [0, 1, 5, 10, 20, width]:
        F1=[]
        MCC=[]
        ZL = []
        for eps in [0, 1, 10, 20, 30, 40, 50]:
            file_name=f"{dataset}_{image_type}_{str(width)}_{model_name}_{str(run_id)}_parameters_{str(delta)}_{str(eps)}"
            if dataset=="lilac" and multi_dimension==False :
                file_name = file_name+"_multi-dimension-norm"
            #print(f" load data for {file_name}")
            f1  = np.load("../results/"+file_name+"_f1.npy")
            zl  = np.load("../results/"+file_name+"_z_one.npy")
            mcc  = np.load("../results/"+file_name+"_mcc.npy")
            if delta==10 and eps >0:
                F1.append(f1[0]*100)
                MCC.append(mcc[0])
                ZL.append(zl[0])

        #plt.plot([1, 10, 20, 30, 40, 50], np.array(MCC) , label = f'$\delta=${delta}', marker='v',markersize=5)
        if delta==10:
            results_mcc[delta]=MCC
            results_f1[delta]=F1
            results_zl[delta]=ZL
    return results_f1


In [ ]:
results_delta={}
for dataset in ["plaid", "lilac"]:
    results_f1=get_param_results(dataset=dataset)
    results_delta[dataset]=np.hstack(list(results_f1.values()))


In [ ]:
fig=figure(columns=1)
plt.plot(np.array([1, 10, 20, 30, 40, 50]), results_delta["plaid"], marker='v',markersize=3, label="PLAID with $\delta=10$", color=colors[0])
plt.plot(np.array([1, 10, 20, 30, 40, 50]), results_delta["lilac"], marker='v',markersize=3, label="LILAC with $\delta=10$", color="darkorange", alpha=0.8)
plt.ylim(70, 100)
plt.xlabel('$\lambda$', fontsize=12)
plt.ylabel('$F_1$ macro $(\%)$', fontsize=12)
ax = plt.gca()
leg = ax.legend(fontsize=10)
savefig(fig_path+f"paramaeter_delta_10", format=".pdf")

### Delta when eps=10

In [ ]:
def get_param_results(dataset="lilac"):
    model_name='CNN'
    width=50
    run_id=1
    cv=4
    multi_dimension=True
    isc=False
    names=lilac_names   if dataset=="lilac"   else plaid_names  
    dataset = dataset+"_isc" if isc else dataset
    image_type= "adaptive"
    baseline = False
    results_cm = {}
    results_f1={}
    results_mcc={}
    results_zl={}
    preds = {}
    trues = {}
    currents = {}
    for eps in [0, 1, 10, 20, 30, 40, 50]:
        F1=[]
        MCC=[]
        ZL = []
        for delta in [0, 1, 5, 10, 20, width]:
            file_name=f"{dataset}_{image_type}_{str(width)}_{model_name}_{str(run_id)}_parameters_{str(delta)}_{str(eps)}"
            if dataset=="lilac" and multi_dimension==False :
                file_name = file_name+"_multi-dimension-norm"
            #print(f" load data for {file_name}")
            f1  = np.load("../results/"+file_name+"_f1.npy")
            zl  = np.load("../results/"+file_name+"_z_one.npy")
            mcc  = np.load("../results/"+file_name+"_mcc.npy")
            if eps==10:
                F1.append(f1[0]*100)
                MCC.append(mcc[0])
                ZL.append(zl[0])

        #plt.plot([1, 10, 20, 30, 40, 50], np.array(MCC) , label = f'$\delta=${delta}', marker='v',markersize=5)
        if eps==10:
            results_mcc[eps]=MCC
            results_f1[eps]=F1
            results_zl[eps]=ZL
    return results_f1

In [ ]:
results_delta={}
for dataset in ["plaid", "lilac"]:
    results_f1=get_param_results(dataset=dataset)
    results_delta[dataset]=np.hstack(list(results_f1.values()))
fig=figure(columns=1)
plt.plot(np.array([0, 1, 5, 10, 20, 50]), results_delta["plaid"], marker='v',markersize=3, label="PLAID with $\lambda=10$", color=colors[0])
plt.plot(np.array([0, 1, 5, 10, 20, 50]), results_delta["lilac"], marker='v',markersize=3, label="LILAC with $\lambda=10$", color="darkorange", alpha=0.8)
plt.ylim(70, 100)
plt.xlabel('$\delta$', fontsize=12)
plt.ylabel('$F_1$ macro $(\%)$', fontsize=12)
ax = plt.gca()
leg = ax.legend(fontsize=10)
savefig(fig_path+f"paramaeter_lambda_10", format=".pdf")    

### Embending size

In [ ]:
def get_param_width_results(dataset="lilac"):
    model_name='CNN'
    width=50
    run_id=1
    cv=4
    multi_dimension=True
    isc=False
    names=lilac_names   if dataset=="lilac"   else plaid_names  
    dataset = dataset+"_isc" if isc else dataset
    image_type= "adaptive"
    baseline = False
    
    results_f1={}
    results_mcc={}
    results_zl={}
    results_time={}
    preds = {}
    trues = {}
    currents = {}
    delta=10
    eps=10
    widths = [30, 50, 60, 80, 100]
    results = pd.DataFrame( index=widths, columns=["F1", "MCC", "ZL", "TIME"])
    for width in widths:
        file_name=f"{dataset}_{image_type}_{str(width)}_{model_name}_{str(run_id)}_parameters_emb_size_{str(delta)}_{str(eps)}"
        if dataset=="lilac" and multi_dimension==False :
            file_name = file_name+"_multi-dimension-norm"
           
        f1  = np.load("../results/"+file_name+"_f1.npy").flatten()
        zl  = np.load("../results/"+file_name+"_z_one.npy").flatten()
        mcc  = np.load("../results/"+file_name+"_mcc.npy").flatten()
        time = np.load("../results/"+file_name+"_time.npy").flatten()
        
        results_f1[width]=list(f1)[0]
        results_mcc[width]=list(mcc)[0]
        results_zl[width]=list(zl)[0]
        results_time[width]=list(time)[0]/60
        
    results["F1"] = pd.Series(results_f1)
    results["MCC"] = pd.Series(results_mcc)
    results["ZL"] = pd.Series(results_zl)
    results["TIME"] = pd.Series(results_time)
    results=results.round(4)
    return results
           
results={}
for dataset in ["plaid", "lilac"]:
    results[dataset]=get_param_width_results(dataset=dataset)

In [ ]:
fig=figure(columns=1)
plt.plot(np.array([30, 50, 60, 80, 100]), results['plaid']['F1'].values*100, marker='v',markersize=3, label="PLAID ",color=colors[0])
plt.plot(np.array([30, 50, 60, 80, 100]), results['lilac']['F1'].values*100, marker='v',markersize=3, label="LILAC ",color="darkorange", alpha=0.8)
plt.ylim(70, 100)
plt.xlabel('$w$', fontsize=12)
plt.ylabel('$F_1$ macro $(\%)$', fontsize=12)
ax = plt.gca()
leg = ax.legend(fontsize=10)
savefig(fig_path+f"paramaeter_width", format=".pdf")

In [ ]:
fig=figure(columns=1)
plt.plot(np.array([30, 50, 60, 80, 100]), results['plaid']['TIME'].values, marker='v',markersize=3, label="PLAID ", color=colors[0])
plt.plot(np.array([30, 50, 60, 80, 100]), results['lilac']['TIME'].values, marker='v',markersize=3, label="LILAC ", color="darkorange", alpha=0.8)
#plt.ylim(70, 100)
plt.xlabel('$w$', fontsize=12)
plt.ylabel('Time ($m$)', fontsize=12)
ax = plt.gca()
ax = plt.gca()
leg = ax.legend(fontsize=10)
savefig(fig_path+f"paramaeter_width_time", format=".pdf")